In [1]:
#imports the basic data science lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# import display module
from IPython.display import display
#import stratify kfold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# import svc,knn    
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
# nlp 
from nltk.stem import WordNetLemmatizer,PorterStemmer
import re
import gensim
# https://github.com/alexandres/lexvec#pre-trained-vectors
# parameter tuning
import optuna
#import typing
from typing import List, Dict, Tuple, Set, Union, Optional, Callable, Any


In [ ]:
#Reads the data from the csv file
x_train = pd.read_csv("./Dataset/x_train.csv",header=None,names=['website','text'])
y_train = pd.read_csv("./Dataset/y_train.csv",header=None,names=['positive'])
x_test = pd.read_csv("./Dataset/x_test.csv",header=None,names=['website',"text"])
y_test = pd.read_csv("./Dataset/y_test.csv",header=None,names=['positive'])

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
display(x_train.head())
display(y_train.head())

In [ ]:
#create a text pre processing class


class TextPreprocessing:
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()
        self.bag_of_words = []
        self.word_counts = {}
        pass
    def text_cleaning(self,text:str):

        re_s = [
            #remove the html tags
            (r'<.*?>',''),
            #remove the urls
            (r'http\S+|www.\S+',''),
            #remove the emails
            (r'\S+@\S+',''),
            #remove the new line
            (r'\n',''),
            #remove the special characters
            (r'[^\w\s]',''),
            #remove the numbers
            (r'\d+',''),
            #remove the stop words
            (r'\b\w{1,2}\b',''),
            #remove the extra spaces
            (r'\s+',' ')
        ] 

        for regex in re_s:
            text = re.sub(regex[0],regex[1],text)
        #convert the text to lower case
        text = text.lower()
        return text
    def text_stemming(self,text:str):
        #create the stemmer object
        stemmer = self.stemmer
        #stem the text
        text = " ".join([stemmer.stem(word) for word in text.split()])
        return text
    def text_lemmatization(self,text:str):
        #create the lemmatizer object
        lemmatizer = self.lemmatizer
        text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
        return text

    def text_tokenization(self,text:str):
        #tokenize the text
        text = text.split()
        return text
    def text_bag_of_words(self,text:str):
        #create the bag of words
        token = self.text_tokenization(text)
        self.bag_of_words.extend(token)
        self.bag_of_words = list(set(self.bag_of_words))
        self.text_word_counts(token)
        return self.bag_of_words
    def text_word_counts(self,token:List[str]):
        #create the word counts
        # text = self.text_tokenization(text)
        for word in token:
            if word in self.word_counts:
                self.word_counts[word] += 1
            else:
                self.word_counts[word] = 1
        return self.word_counts
    def __call__(self,text:str,bow:bool=True):
        text = self.text_cleaning(text)
        text = self.text_lemmatization(text)
        if bow:
            self.text_bag_of_words(text)

        return text
    def get_bag_of_words(self):
        return self.bag_of_words,self.word_counts
# define stratify k fold
def stratified_kfold_cross_validation(x_train,y_train,clf,n_splits=3,):
    skf = StratifiedKFold(n_splits=n_splits,random_state=42,shuffle=True)
    evals = []
    for train_index, test_index in skf.split(x_train["vec"],y_train["positive"]):
        _x_train,_x_val = x_train["vec"].iloc[train_index],x_train["vec"].iloc[test_index]
        _y_train,_y_val = y_train["positive"].iloc[train_index],y_train["positive"].iloc[test_index]
        clf.fit(_x_train,_y_train)
        y_pred = clf.predict(_x_val)
        y_true = _y_val
        evals.append({
            "accuracy":accuracy_score(y_true,y_pred),
            "f1_score":f1_score(y_true,y_pred),
            "precision":precision_score(y_true,y_pred),
            "recall":recall_score(y_true,y_pred)
        })
    return evals


In [ ]:
t_preprocess = TextPreprocessing()
model = gensim.models.KeyedVectors.load_word2vec_format('./models/lexvec.enwiki+newscrawl.300d.W.pos.vectors.gz', binary=False)

In [ ]:
##define pipeliens for the text preprocessing
def vectorize_text(text):
    #word2vec
    vector = np.zeros(300)
    for word in text.split():
        if word in model:
            vector += model[word]
    return vector
def text_preprocessing_pipeline(df,t_preprocess:TextPreprocessing,bow=True):
    df['processed'] = df['text'].apply(lambda x: t_preprocess(x,bow=bow))

    
    keys = [("vec","vec"+str(i)) for i in range(300)]
    vec_df = pd.DataFrame(columns=keys)
    df = df.join(vec_df)
    df.columns=pd.MultiIndex.from_tuples([('website',"value"),('text','value'),('processed','value')]+keys)
    df[keys] = pd.DataFrame(df[('processed','value')].apply(lambda x: vectorize_text(x)).tolist(), index= df.index)
    #create multi index so vector columns can be accessed easily
    return df

x_train = text_preprocessing_pipeline(x_train,t_preprocess)
x_train.head()

In [ ]:
bow, counts = t_preprocess.get_bag_of_words()
print("bag of words counts",len(bow))

In [ ]:
# define models to train on
models = [
    ("Random Forest",RandomForestClassifier(n_estimators=100,random_state=42)),
    ("Logistic Regression",LogisticRegression(random_state=42,solver='lbfgs', max_iter=1000)),
    ("SVM",SVC(random_state=42)),
    ("KNN",KNeighborsClassifier()),
]

In [ ]:
# do stratified k fold cross validation on the models
evals = []
for name,clf in models:
    _eval = stratified_kfold_cross_validation(x_train,y_train,clf,n_splits=3)
    _eval = pd.DataFrame(_eval)
    _eval["modelname"] = name
    evals.append(_eval)
#concat the evals
eval_df = pd.concat(evals)
results_df = eval_df.groupby("modelname").mean()
results_df.head()

In [ ]:
#get the best results
results_df.idxmax()

In [ ]:
#parameter tuning for SVM using optuna
def objective(trial):
    #define the parameters to tune
    params = {
        "C":trial.suggest_float("C",1e-10,1e10),
        "kernel":trial.suggest_categorical("kernel",["linear","rbf"]),
        "gamma":trial.suggest_categorical("gamma",["scale","auto"]),
        "degree":trial.suggest_int("degree",1,5),
        "coef0":trial.suggest_float("coef0",1e-10,1e10)
    }
    #define the model
    clf = SVC(**params,random_state=42)
    #do stratified k fold cross validation
    evals = stratified_kfold_cross_validation(x_train,y_train,clf,n_splits=3)
    evals = pd.DataFrame(evals)
    #get the mean of the evals
    return evals["accuracy"].mean()
#optimize the model
study = optuna.create_study(direction="maximize")
study.optimize(
    objective,
    n_trials=50,#300 seconds or 50 trials whichever comes first
    timeout=300,#300 seconds or 50 trials whichever comes first
    # n_jobs=-1,# use all core
    gc_after_trial=True, #garbage collect after each trial (free up memory)
    show_progress_bar=True
    )

In [ ]:
#train the best model on the whole dataset
clf = SVC(random_state=42)
clf.fit(x_train["vec"],y_train["positive"])

#test the model on the test set
_x_test = text_preprocessing_pipeline(x_test,t_preprocess,bow=False)
y_pred = clf.predict(_x_test["vec"])
y_true = y_test["positive"]


accuracy = accuracy_score(y_true,y_pred)
f1_score = f1_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)
confusion_mat = confusion_matrix(y_true,y_pred)
print("accuracy",accuracy)
print("f1_score",f1_score)
print("precision",precision)
print("recall",recall)
print("confusion matrix",confusion_mat)


